Depositos para enviar, orderm de preferencia:
musical_matriz
musical_filal
em seguida onde houver mais estoque


onde days_available = 0 criar observação onde informa que há muito tempo sem estoque

In [1]:
import json
import os
from pandas import json_normalize
import psycopg2
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN_MCENTER = os.getenv("ACCESS_TOKEN_MCENTER")
ACCESS_TOKEN_BUENOSHOPS = os.getenv("ACCESS_TOKEN_BUENOSHOPS")
ACCESS_TOKEN_MUSICALCRIS = os.getenv("ACCESS_TOKEN_MUSICALCRIS")

HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

SMARTGO_TOKEN = os.getenv("SMARTGO_TOKEN")

# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [2]:
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

In [3]:
def get_wms_data(SMARTGO_TOKEN):
    try:
        url = "https://apigateway.smartgo.com.br/estoque/saldo"

        payload = {}
        headers = {"api_key": SMARTGO_TOKEN}

        response = requests.request("GET", url, headers=headers, data=payload)

        if response.status_code == 200:
            data = json.loads(response.text)

            items = data.get("model", {}).get("items", [])

            result_list = []

            for item in items:
                result_dict = {
                    "id_depositante": item.get("idDepositante"),
                    "depositante": item.get("depositante"),
                    "area": item.get("area"),
                    "areaComputaSaldo": item.get("areaComputaSaldo"),
                    "idProduto": item.get("idProduto"),
                    "produto_nome": item.get("produtoNome"),
                    "produtoCodigoInterno": item.get("produtoCodigoInterno"),
                    "produtoCodigoExterno": item.get("produtoCodigoExterno"),
                    "quantidade": item.get("quantidade"),
                    "quantidadeProduto": item.get("quantidadeProduto"),
                    "quantidadeDeMovimentacao": item.get("quantidadeDeMovimentacao"),
                    "quantidadeProdutosEmbalagem": item.get(
                        "quantidadeProdutosEmbalagem"
                    ),
                    "tipoUnidadeEmbalagem": item.get("tipoUnidadeEmbalagem"),
                    "tipoUnidadeMovimentacao": item.get("tipoUnidadeMovimentacao"),
                    "tipoUnidadeProduto": item.get("tipoUnidadeProduto"),
                    "quantidadeEnderecos": item.get("quantidadeEnderecos"),
                    "quantidade_disponivel": item.get("quantidadeDisponivel"),
                    "quantidadeEmExpedicao": item.get("quantidadeEmExpedicao"),
                    "pedidosCodigosExternos": item.get("pedidosCodigosExternos"),
                    "codigosDeIdentificacao": item.get("codigosDeIdentificacao"),
                    "notasFiscais": item.get("notasFiscais"),
                    "depositos": item.get("depositos"),
                }
                result_list.append(result_dict)

            df = pd.DataFrame(result_list)
            cols = [
                "id_depositante",
                "depositante",
                "idProduto",
                "produto_nome",
                "produtoCodigoInterno",
                "produtoCodigoExterno",
                "quantidade_disponivel",
            ]
            df = df[cols]
            return df

    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
        return None

    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

### Período a consultar

In [4]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 12, 1).date()
data_fim = datetime(2024, 1, 4).date()
# data_fim = data_fim + timedelta(days=1)  # + 1 dia para pegar a data atual no DB
# data_fim = data_fim - timedelta(days=1)  # + 1 dia para pegar a data atual no DB
print("data_inicio: ", data_inicio)
print("data_fim: ", data_fim)

data_inicio:  2023-12-01
data_fim:  2024-01-04


### Historico de estoque fulfillment

In [5]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim};'"
    # sql_query = f"SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '2023-12-04' AND '2023-12-05';"
    print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '2023-12-01' AND '2024-01-04;'


C:\Users\couti\AppData\Local\Temp\ipykernel_18520\1935300295.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [6]:
# df_stock
# print(df_stock.shape)
# duplicates = df_stock[df_stock.duplicated(keep=False)]
# duplicates

In [7]:
# # datas consultadas, dias em que um produto pode ou não estar disponível
# # df_stock["created_at"].value_counts().index.to_list()
# df_stock["created_at"].value_counts().sort_index()
# df_stock["created_at"].dt.date.value_counts().sort_index()
# # df_stock["created_at"].dt.date.value_counts()

# # len(df_stock["created_at"].dt.date.value_counts().sort_index())

In [8]:
# df_stock

In [9]:
# # buscando anuncio e verificando dias consultados
# result = df_stock[df_stock['ml_inventory_id'] == 'FBAY80623']
# result = result.sort_values(by='created_at')
# result

In [10]:
# # encontrando anuncios pela data
# df_stock["created_at"] = pd.to_datetime(df_stock["created_at"])
# result = df_stock.loc[df_stock['created_at'].dt.date == pd.to_datetime('2023-12-7').date()]
# result

In [11]:
# df_stock[df_stock['created_at']=='2023-12-07 03:01:03.036386']

### Ordenando stock por data

In [12]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by="created_at", ascending=False)
df_stock["data"] = df_stock["created_at"].dt.date
df_stock = df_stock.drop(["created_at"], axis=1)

In [13]:
# df_stock

In [14]:
# df_stock.dtypes

In [15]:
# df_stock['data'] = pd.to_datetime(df_stock['data'])

# dfx = condf(df_stock, 'data', '2023-11-1')
# dfx

In [16]:
# # A linha abaixo conta as ocorrências de cada valor em 'ml_inventory_id'
# # value_counts = result['ml_inventory_id'].value_counts()
# value_counts = df_stock['ml_inventory_id'].value_counts()

# # Filtra para incluir apenas aqueles que ocorrem mais de 2 vezes
# filtered_counts = value_counts[value_counts > 1]

# filtered_counts.shape
# # filtered_counts
# value_counts

### Cria coluna has_stock

In [17]:
# df_stock[df_stock["detail_status"] == "transfer"]

In [18]:
# condf(df_stock,'ml_inventory_id','CRMK90073')

In [19]:
# condf(df_stock, "detail_status", "transfer")

In [20]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by="data", ascending=False).reset_index(drop=True)

In [21]:
# df_stock

In [22]:
# condf(df_stock, "ml_inventory_id", "DSGP06967")

In [23]:
# print(df_stock.shape)
# df_stock = df_stock.drop_duplicates()
# print(df_stock.shape)

#### Dias em que produto esteve disponível

In [24]:
# condf(df_stock,'ml_inventory_id','ABCB20467')

In [25]:
## Contando dias em que produto esteve disponível
days_available = df_stock.groupby(["ml_inventory_id"])["has_stock"].sum().reset_index()
days_available = days_available.rename(columns={"has_stock": "days_available"})

In [26]:
# days_available

In [27]:
# days_available.sort_values(by='days_available',ascending=False)
# days_available['days_available'].value_counts()

# ## Contando dias em que produto esteve disponível
# days_available = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available = days_available.rename(columns={"has_stock": "days_available"})
# days_available

In [28]:
# condf(df_stock, "ml_inventory_id", "MENA66694")

In [29]:
# ## Contando dias em que produto esteve disponível
# days_available_ = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available_ = days_available_.rename(columns={"has_stock": "days_available"})
# days_available_

In [30]:
# condf(days_available, "ml_inventory_id", "DSGP06967")
# condf(days_available, "ml_inventory_id", "DSGP06967")

In [31]:
# Unindo DFs
df_stock_days = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# df_stock_days_left = df_stock.merge(days_available, on=["ml_inventory_id"], how="left")
# df_stock.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# df_stock = df_stock.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# Verificando o formato após a correção
# print(df_stock.shape)
#

In [32]:
# print(df_stock.shape)
# print(df_stock_days.shape)
# print(df_stock_days_left.shape)
# df_stock_days.sample(3)

In [33]:
# # Unindo DFs
# df_stock_ = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# # df_stock_.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# # df_stock_ = df_stock_.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# # Verificando o formato após a correção
# print(df_stock_.shape)
# df_stock_

In [34]:
# data de hoje
data_de_hoje = datetime.now().date()
data_de_hoje = data_de_hoje - timedelta(days=1)
# print(data_de_hoje)

df_stock_days["data"] = pd.to_datetime(df_stock_days["data"])

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock_days[df_stock_days["data"].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(
    columns={"available_quantity": "available_quantity_today"}
)

# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [35]:
df_stock_today

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available,total_available_quantity
0,WQLG77218,5,transfer,1.0,MLB1942106803,nan,2024-01-03,True,1,6
34,NOMF53205,1,None,NaN,MLB1629586589,nan,2024-01-03,True,34,1
68,HGRL10383,2,None,NaN,MLB2003270402,nan,2024-01-03,True,32,2
102,BPGQ10723,12,noFiscalCoverage,12.0,MLB2001140622,nan,2024-01-03,True,23,12
136,KBOK48388,2,None,NaN,MLB1459771919,51818479775,2024-01-03,True,34,2
...,...,...,...,...,...,...,...,...,...,...
12206,CDBO10673,8,None,NaN,MLB2006338750,nan,2024-01-03,True,34,8
12240,NZGA07022,2,None,NaN,MLB2006323551,nan,2024-01-03,True,31,2
12274,PMPX06029,8,None,NaN,MLB2006353434,nan,2024-01-03,True,34,8
12308,VUJI10625,13,None,NaN,MLB2006357183,nan,2024-01-03,True,34,13


In [36]:
# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [37]:
# df_stock_today.sample(5)

In [38]:
# condf(df_stock,'ml_inventory_id','FSNB97805')
# condf(df_stock_today,'ml_inventory_id','FSNB97805')

### Buscando hitorico de orders no BD

In [39]:
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM mcenter_ml_orders WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# # Ordenando orders por data
# df_orders = df_orders.sort_values(by="date_approved", ascending=False)
# df_orders["data"] = df_orders["date_approved"].dt.date
# df_orders = df_orders.drop(["date_closed","date_approved"], axis=1)

# filtros
df_orders = df_orders[df_orders["logistic_type"] == "fulfillment"]
# df_orders = df_orders.drop(columns=['category_id','pack_id','variation_attributes_id','variation_name','variation_value_id', 'data'])
df_orders = df_orders.drop(
    columns=[
        "category_id",
        "pack_id",
        "variation_attributes_id",
        "variation_name",
        "variation_value_id",
    ]
)

# change column
df_orders["variation_id"] = df_orders["variation_id"].replace("nan", "0", regex=True)

# df_orders = df_orders[df_orders["order_status"] == "paid"]
# df_orders = df_orders[df_orders["payment_status"] == "approved"]


# df_orders = df_orders.drop(
#     ["pack_id", "date_approved", "order_status", "payment_status"], axis=1
# )
df_orders.rename(columns={"quantity": "sold_quantity"}, inplace=True)

SELECT * FROM mcenter_ml_orders WHERE date_closed BETWEEN '2023-12-01' AND '2024-01-04'


C:\Users\couti\AppData\Local\Temp\ipykernel_18520\2680549919.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql(sql_query, conn)


In [40]:
print(df_orders.shape)
df_orders = df_orders.drop_duplicates()
print(df_orders.shape)

(3963, 13)
(3648, 13)


In [41]:
# Ordenando orders por data
df_orders = df_orders.sort_values(by="date_approved", ascending=False)
df_orders["data"] = df_orders["date_approved"].dt.date
df_orders = df_orders.drop(["date_closed", "date_approved"], axis=1)

In [42]:
df_orders.sample(3)

,ml_code,variation_id,seller_sku,order_id,sold_quantity,title,order_status,payment_status,shipping_id,variation_value_name,logistic_type,data
4229,MLB2192269456,174349843073,FULLMEMPHISHONUSG,2000007216569046,1,Ukulele Soprano Memphis Tagima 21 Havaí Honu ...,paid,approved,42937221033,Verde,fulfillment,2023-12-21
5416,MLB3461718736,0,FULLP10LNINJA20FT,2000007296773092,1,Cabo Para Guitarra Violão Baixo Santo Angelo N...,paid,approved,42972527752,None,fulfillment,2024-01-03
398,MLB3427769549,0,FULLIZZO6480,2000007047779174,1,Pandeiro Profissional Izzo Pele Preta 10 Corpo...,paid,approved,42865149295,None,fulfillment,2023-12-03


In [43]:
# df_orders.head()

In [44]:
# df_orders
# df_orders['pack_id'].value_counts()

# # condf(df_orders,'pack_id','2000005059931419')

In [45]:
# print(df_orders.shape)
# df_orders.head(3)

In [46]:
# condf(df_orders, "ml_code", "MLB1992541482")

#### Total de vendas por ml_code e variation_id

In [47]:
# df_orders.shape

In [48]:
# df_orders

In [49]:
# condf(df_orders,'ml_code','MLB3516894673')

In [50]:
# Total de vendas por ml_code e id de variação
df_orders_quantity = (
    df_orders.groupby(["ml_code", "variation_id"])["sold_quantity"].sum().reset_index()
)

In [51]:
df_orders_quantity

,ml_code,variation_id,sold_quantity
0,MLB1363602423,0,13
1,MLB1400355425,0,30
2,MLB1401946009,0,7
3,MLB1403147752,48946752091,1
4,MLB1403763907,0,5
...,...,...,...
272,MLB4200104102,0,1
273,MLB4202595608,0,44
274,MLB4217370456,0,1
275,MLB4237823132,0,9


In [52]:
print(f"Número de ml_code únicos: {len(df_orders_quantity['ml_code'].unique())}")
print(
    f"Número de variation_id únicos: {len(df_orders_quantity['variation_id'].unique())}"
)

Número de ml_code únicos: 223
Número de variation_id únicos: 98


In [53]:
# condf(df_orders_quantity,'ml_code','MLB2075516288')
# df_orders_quantity['ml_code'].value_counts()

In [54]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(
    # df_orders, resultado, on=["ml_code", "variation_id"], how="inner"
    df_orders,
    df_orders_quantity,
    on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales = df_total_sales.rename(
    columns={"sold_quantity_y": "total_sold_quantity"}
)
df_total_sales = df_total_sales.drop(
    columns=["sold_quantity_x", "order_status", "payment_status"]
)

In [55]:
print(f"Total de vendas = {df_total_sales.shape[0]}")

Total de vendas = 3648


In [56]:
# df_orders.shape

In [57]:
# df_total_sales.columns

In [58]:
# df_total_sales.sample()

In [59]:
# # # Exibir todas as linhas e colunas
# # pd.set_option('display.max_rows', None)
# # pd.set_option('display.max_columns', None)

# # Voltar ao normal
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')


# # condf(df_total_sales,'ml_code','MLB3427769549')
# # # x = condf(df_orders,'ml_code','MLB3427769549')

# # # x = x.sort_values(by='data')
# # # x.head(15)
# # x = df_orders[df_orders['ml_code']=='MLB3427769549']
# # x['data'] = pd.to_datetime(x['data'])
# # x[x['data']=='2023-11-22']
# # # x['sold_quantity'].value_counts()

# # # counts = x[x['sold_quantity'] > 1]['sold_quantity'].value_counts()
# # # counts

# # y = x.groupby('data')['sold_quantity'].sum().reset_index()
# # y

# # # # x

In [60]:
# x.dtypes

In [61]:
# df_total_sales['data'].value_counts()

In [62]:
# condf(df_total_sales, "ml_code", "MLB4127982748")
# condf(df_total_sales, "ml_code", "MLB4127982748")

In [63]:
# df_total_sales = df_total_sales.drop(["sold_quantity_x", "shipping_id", "data"], axis=1)
# df_total_sales = df_total_sales.drop_duplicates()

In [64]:
# condf(df_total_sales, "ml_code", "MLB1992541482")

Neste ponto temos o total de itens vendidos de um anúncio por período e a quantidade de dias em que um produto esteve disponível.
precisamos juntar esses dados para calcular, para isso trarei as informações de produtos

In [65]:
# print(df_total_sales.shape)
# df_total_sales.sample()

In [66]:
# condf(df_total_sales, 'ml_code', 'MLB3461718736')
# condf(df_orders, 'ml_code', 'MLB3461718736')
# # condf(resultado, 'ml_code', 'MLB3427769549')
# # condf(df_stock, 'ml_code', 'MLB3427769549')

In [67]:
# print(df_stock_today.shape)
# df_stock_today.sample()

#### Buscando Produtos

In [68]:
# Buscando dados de produtos na tabela tiny_fulfillment
try:
    conn = psycopg2.connect(**db_config)
    sql_query = "SELECT * FROM mcenter_items"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar tiny_fulfillment: {e}")

except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

# df_codes["ml_code"] = df_codes["ml_code"].apply(lambda x: "MLB" + str(x))
df_codes.rename(columns={"inventory_id": "ml_inventory_id"}, inplace=True)
df_codes = df_codes.drop(["created_at", "updated_at"], axis=1)

C:\Users\couti\AppData\Local\Temp\ipykernel_18520\3854357628.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


### separando itens que são catálogos dos que não são

In [69]:
df_not_catalogo = df_codes[df_codes["catalog_listing"] == False]
df_catalogo = df_codes[df_codes["catalog_listing"] == True]

In [70]:
df_not_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
252,MLB2001160804,SJLM76488,NaN,NaN,active,False,fulfillment


In [71]:
df_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
410,MLB3286467401,YAZB38863,NaN,NaN,active,True,fulfillment


In [72]:
df_not_catalogo

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
0,MLB1435852259,TLCG38735,NaN,NaN,active,False,fulfillment
1,MLB1435836214,PUOF38682,NaN,NaN,active,False,fulfillment
2,MLB1400355425,ZXFT21237,NaN,NaN,active,False,fulfillment
3,MLB1440291959,JGRW38703,NaN,NaN,active,False,fulfillment
4,MLB1564998298,OVRP11252,NaN,NaN,paused,False,fulfillment
...,...,...,...,...,...,...,...
527,MLB1490210934,SPKV63227,nan,NaN,paused,False,fulfillment
528,MLB1556975175,DZVJ05686,Marrom,57895096313,active,False,fulfillment
529,MLB950338822,MAEG54107,nan,NaN,paused,False,fulfillment
530,MLB1662970458,UIUW89800,Preto,64498487033,paused,False,fulfillment


In [73]:
print(df_not_catalogo.shape)
print(df_catalogo.shape)

(361, 7)
(171, 7)


In [74]:
df_catalogo

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
19,MLB2170026503,ZNFW24350,NaN,NaN,paused,True,fulfillment
26,MLB3435374559,RYWV54124,NaN,NaN,paused,True,fulfillment
27,MLB3435352761,QYGO52123,NaN,NaN,paused,True,fulfillment
28,MLB3435350955,HGVR43626,NaN,NaN,active,True,fulfillment
29,MLB3435245361,PKNN52962,NaN,NaN,paused,True,fulfillment
...,...,...,...,...,...,...,...
519,MLB4127982748,RLJX42857,nan,NaN,paused,True,fulfillment
520,MLB3497394073,XGSW66673,nan,NaN,paused,True,fulfillment
521,MLB4191526348,LXOO59456,nan,NaN,paused,True,fulfillment
522,MLB3521778415,PSMV02965,nan,NaN,paused,True,fulfillment


In [75]:
# df_orders.sample()

In [76]:
df_total_sales_cat = pd.merge(
    df_catalogo,
    df_total_sales,
    left_on=["ml_code"],
    right_on=["ml_code"],
    how="left",
)
df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_not_cat = pd.merge(
    df_not_catalogo,
    df_total_sales,
    left_on=["ml_code", "variation_id"],
    right_on=["ml_code", "variation_id"],
    how="left",
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates()

In [77]:
df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].fillna(0)
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].fillna(0)

df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].astype("int64")
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].astype("int64")

In [78]:
# df_total_sales_cat.sample(5)
# df_total_sales_not_cat['total_sold_quantity'].value_counts()
# df_total_sales_cat['total_sold_quantity'].value_counts()

In [79]:
print(df_total_sales_cat.shape)
print(df_total_sales_not_cat.shape)

(2083, 16)
(1470, 15)


In [80]:
# df_total_sales_cat

In [81]:
# df_total_sales_cat = df_total_sales_cat.drop(
#     columns=["data", "shipping_id", "variation_id_x", "order_id"]
# )
df_total_sales_cat_x = df_total_sales_cat.drop(
    columns=["data", "shipping_id", "variation_id_x", "order_id"]
)
df_total_sales_cat_x = df_total_sales_cat_x.drop_duplicates()
df_total_sales_cat_x.shape

(172, 12)

In [82]:
# df_total_sales_cat_x

In [83]:
# df_total_sales_cat['value_name'].value_counts()

In [84]:
# condf(df_total_sales_cat, 'ml_code','MLB3427769549')

In [85]:
# df_vendas_total_sales_not_cat

In [86]:
print(df_total_sales_not_cat.shape)

(1470, 15)


In [87]:
print(df_total_sales_not_cat.shape)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop(
    columns=["data", "shipping_id", "order_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat_x.drop_duplicates()
print(df_total_sales_not_cat_x.shape)

(1470, 15)
(361, 12)


In [88]:
# df_total_sales_cat_x

In [89]:
print(df_total_sales_cat_x.shape)
print(df_total_sales_not_cat_x.shape)

(172, 12)
(361, 12)


In [90]:
# df_total_sales_cat['ml_inventory_id'].value_counts()

In [91]:
# Antes de desabilitar
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# Seu código para exibir o DataFrame

# # Depois de desabilitar
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')


# df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_cat_x = df_total_sales_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)

In [92]:
# df_orders.shape

In [93]:
# condf(df_total_sales_cat,'ml_inventory_id','WTHI04163')
# condf(y,'ml_inventory_id','WTHI04163')

In [94]:
# condf(df_total_sales_cat,'ml_inventory_id','AWAZ99732')

In [95]:
df_total_cat = df_total_sales_cat_x.copy()
df_total_not_cat = df_total_sales_not_cat_x.copy()

In [96]:
df_total_cat.columns

Index(['ml_code', 'ml_inventory_id', 'value_name', 'variation_id_x', 'status',
       'catalog_listing', 'logistic_type_x', 'variation_id_y', 'seller_sku',
       'order_id', 'title', 'shipping_id', 'variation_value_name',
       'logistic_type_y', 'data', 'total_sold_quantity'],
      dtype='object')

In [97]:
# Somando total de vendas por inventory_id
df_sum_qt_sold_cat = (
    df_total_cat.groupby("ml_inventory_id")["total_sold_quantity"].sum().reset_index()
)
df_sum_qt_sold_cat = df_sum_qt_sold_cat.rename(
    columns={"total_sold_quantity": "total_sold_catalog"}
)

df_total_cat = pd.merge(
    df_total_cat,
    df_sum_qt_sold_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)

df_total_cat.rename(columns={"variation_id_y": "variation_id_"})


df_sum_qt_sold_not_cat = (
    df_total_not_cat.groupby("ml_inventory_id")["total_sold_quantity"]
    .sum()
    .reset_index()
)
df_sum_qt_sold_not_cat = df_sum_qt_sold_not_cat.rename(
    columns={"total_sold_quantity": "total_sold_not_catalog"}
)
df_total_not_cat = pd.merge(
    df_total_not_cat,
    df_sum_qt_sold_not_cat[["ml_inventory_id", "total_sold_not_catalog"]],
    on="ml_inventory_id",
    how="left",
)

In [98]:
print(df_total_cat.shape)
print(df_total_not_cat.shape)

(170, 17)
(361, 16)


In [99]:
df_total_cat.rename(columns={"variation_id_y": "variation_id"}, inplace=True)
df_total_cat.shape

(170, 17)

In [100]:
# condf(df_total_cat,'ml_inventory_id','WTHI04163')

In [101]:
df_total_cat = df_total_cat.drop_duplicates(subset=["ml_inventory_id"])

# df_total_cat[df_total_cat['ml_inventory_id']=='WTHI04163']

pergar ml_inventory_id e sum_total e acrescesntar essas informações no df de não catalogo pelo ml_inventory_id
ao fim deixar somatório do catalogo e não catalogo


In [102]:
# cols=['ml_inventory_id','total_sold_catalog']
# df_total_cat = df_total_cat[cols]
# df_total_cat.sample()

In [103]:
df_combined = pd.merge(
    df_total_not_cat,
    df_total_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)
# df_combined.sample()

In [104]:
# df_total_not_cat.shape

In [105]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [106]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [107]:
# df_combined['ml_inventory_id'].value_counts()

In [108]:
print(df_combined.shape)

(361, 17)


In [109]:
df = pd.merge(df_combined, df_stock_today, on="ml_inventory_id", how="left")
df["total_sold_catalog"] = df["total_sold_catalog"].fillna(0).astype("int64")

In [110]:
# df_combined

In [111]:
df.shape

(361, 26)

In [112]:
days = 30

df["total_sold"] = df["total_sold_catalog"] + df["total_sold_not_catalog"]
# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df["period_send_fulfillment"] = np.ceil(
    (df["total_sold"] / df["days_available"]) * days - df["total_available_quantity"]
)

df["period_send_fulfillment"] = df["period_send_fulfillment"].fillna(0)

In [113]:
def calculate_percentual_send(row):
    if row["days_available"] != 0:
        if np.ceil(
            (row["total_sold"] / row["days_available"]) * days * 0.7
            > row["total_available_quantity"]
        ):
            # return (np.ceil(row["total_sold"] / row["days_available"]) * days - row["total_available_quantity"])
            return np.ceil(
                (row["total_sold"] / row["days_available"]) * days
                - row["total_available_quantity"]
            )

    return 0

In [114]:
# Aplicando a função à coluna "percentual_send"
df["stock_replenishment"] = df.apply(calculate_percentual_send, axis=1)

In [115]:
# df.sample(5)

In [116]:
# df.columns

In [117]:
# df.shape

In [118]:
# df

In [119]:
df_have_itens = df[df["days_available"] > 0]
# df_have_itens.shape
# df_have_itens

In [120]:
# produtos sem estoque no período
df_no_itens = df[df["days_available"] <= 0]
# df_no_itens = df_no_itens.drop(columns=["period_send_fulfillment"])
# df_no_itens.shape

In [121]:
df_sold_zero = df_have_itens[df_have_itens["total_sold"] == 0]
df_sold = df_have_itens[df_have_itens["total_sold"] > 0]

In [122]:
# print(df_no_itens.shape)
# print(df_sold.shape)
# print(df_sold_zero.shape)

In [123]:
# df_sold_zero.sample(3)

In [124]:
# df_sold.sample(3)

In [125]:
dfx = df_have_itens.copy()

In [126]:
# df_sold_zero.columns

In [127]:
# dfx.columns

In [128]:
cols = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "total_available_quantity",
    "days_available",
    "total_sold_not_catalog",
    "total_sold_catalog",
    "total_sold",
    "period_send_fulfillment",
    "stock_replenishment",
]

df_sold_zero = df_sold_zero[cols]
df_sold = df_sold[cols]
df_no_itens = df_no_itens[cols]

In [129]:
def rename_columns(df):
    return df.rename(
        columns={
            "detail_status": "transfer_status",
            "detail_quantity": "transfer_quantity",
        }
    )


df_sold_zero = rename_columns(df_sold_zero)
df_sold = rename_columns(df_sold)
df_no_itens = rename_columns(df_no_itens)

In [130]:
# df_no_itens.columns

In [131]:
# dfx = dfx.rename(
#     columns={"detail_status": "transfer_status", "detail_quantity": "transfer_quantity"}
# )

In [132]:
# dfx.sample()
# dfx.shape

In [133]:
# dfx

### Agrupando por categoria de produto para enviar

In [134]:
df_itens_to_send = df_sold[df_sold["stock_replenishment"] > 0]
df_itens_to_send = df_itens_to_send[["ml_inventory_id", "stock_replenishment"]]
df_itens_to_send = df_itens_to_send.rename(columns={"ml_inventory_id": "inventory_id"})

df_itens_to_send.shape

(70, 2)

In [135]:
# df_itens_to_send.sample()

In [136]:
# Buscando relação fulfillment X tiny
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM tiny_fulfillment_mcenter"
    print(sql_query)
    df_tiny_fulfillment = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM tiny_fulfillment_mcenter


C:\Users\couti\AppData\Local\Temp\ipykernel_18520\1370134452.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tiny_fulfillment = pd.read_sql(sql_query, conn)


In [137]:
# Buscando categorias dos produtos
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM mcenter_types"
    print(sql_query)
    df_types = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM mcenter_types


C:\Users\couti\AppData\Local\Temp\ipykernel_18520\2421984111.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_types = pd.read_sql(sql_query, conn)


In [138]:
df_types.shape

(361, 2)

In [139]:
# Agrupando itens a enviar com tipo de itens

df_send_types = pd.merge(df_itens_to_send, df_types, on="inventory_id", how="inner")

In [140]:
print(df_send_types.shape)
print(df_itens_to_send.shape)

(70, 3)
(70, 2)


In [141]:
df_send_types

,inventory_id,stock_replenishment,type
0,JGED44177,9.0,Pandeiro
1,GLMK21133,5.0,Pandeiro
2,FXMN50419,70.0,Cabo
3,KRJP31426,4.0,Suporte Sopro
4,HFNJ52229,372.0,Cabo
...,...,...,...
65,RLJX42857,114.0,Pandeiro
66,GPGU38258,3.0,Encordoamento
67,DZVJ05686,20.0,Ukulele
68,MAEG54107,65.0,Cabo


In [142]:
# Identificar todos os tipos únicos
unique_types = df_send_types["type"].unique()

# Criar grupos onde cada grupo contém todas as instâncias associadas a um tipo
result_dfs = []

for unique_type in unique_types:
    type_group = df_send_types[df_send_types["type"] == unique_type]
    result_dfs.append(type_group)

# Exibir os DataFrames resultantes
for i, result_df in enumerate(result_dfs):
    print(f"Grupo {i + 1}:\n{result_df}\n")
#     result_df

Grupo 1:
   inventory_id  stock_replenishment      type
0     JGED44177                  9.0  Pandeiro
1     GLMK21133                  5.0  Pandeiro
5     LKXA87678                  7.0  Pandeiro
8     CRMK90073                 38.0  Pandeiro
22    MAEG89052                 19.0  Pandeiro
44    WOJB37093                 18.0  Pandeiro
56    RKDM61361                 24.0  Pandeiro
57    RXXA58942                 95.0  Pandeiro
58    PPWL59058                 98.0  Pandeiro
61    QOVS42512                135.0  Pandeiro
65    RLJX42857                114.0  Pandeiro
69    UIUW89800               3719.0  Pandeiro

Grupo 2:
   inventory_id  stock_replenishment  type
2     FXMN50419                 70.0  Cabo
4     HFNJ52229                372.0  Cabo
18    HVCS52763                210.0  Cabo
19    RBUB53192                165.0  Cabo
21    JFGN34621                112.0  Cabo
26    QHVP51310                 40.0  Cabo
28    LGAM52077                  8.0  Cabo
29    YWET52700           

In [143]:
# Lista para armazenar os DataFrames resultantes de cada agrupamento
result_dfs_list = []

while any(result_df.shape[0] > 0 for result_df in result_dfs):
    first_rows_dfs = []
    remaining_rows_dfs = []

    for result_df in result_dfs:
        if not result_df.empty:
            # Pega a primeira linha do DataFrame
            first_row_df = result_df.head(1)
            first_rows_dfs.append(first_row_df)

            # Pega as linhas restantes do DataFrame
            remaining_rows_df = result_df.iloc[1:]
            if not remaining_rows_df.empty:
                remaining_rows_dfs.append(remaining_rows_df)
            else:
                print(
                    f"DataFrame vazio encontrado após extrair a primeira linha:\n{result_df}"
                )

    # Adiciona o DataFrame resultante de cada agrupamento à lista
    result_dfs_list.append(pd.concat(first_rows_dfs, ignore_index=True))

    # Atualiza a lista result_dfs com os DataFrames restantes
    result_dfs = remaining_rows_dfs.copy()

# cols = ['inventory_id', 'ml_code', 'seller_sku', 'title', 'tiny_id', 'tiny_sku', 'stock_replenishment', 'qtd_to_send', 'type']
cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "stock_replenishment",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "qtd_to_send",
    "type",
]

# Exibir os DataFrames resultantes de cada agrupamento
for i, result_df in enumerate(result_dfs_list):
    result_df = pd.merge(result_df, df_tiny_fulfillment, on="inventory_id", how="inner")
    result_df = result_df.drop(["Tipo de produto"], axis=1)
    result_df = result_df.rename(
        columns={
            "Quantidade do item": "qtd_item",
            "SKU": "seller_sku",
            "Título do anúncio": "title",
            "ID Tiny": "tiny_id",
            "SKU Tiny": "tiny_sku",
        }
    )

    result_df["qtd_to_send"] = result_df["stock_replenishment"] * result_df["qtd_item"]
    result_df["qtd_item"] = result_df["qtd_item"].astype("int64")
    result_df["qtd_to_send"] = result_df["qtd_to_send"].astype("int64")

    result_df = result_df[cols]
    print(f"Novo DataFrame do Agrupamento {i + 1}:\n", result_df)

DataFrame vazio encontrado após extrair a primeira linha:
   inventory_id  stock_replenishment           type
62    WYFB54549                  8.0  Suporte Corda
DataFrame vazio encontrado após extrair a primeira linha:
   inventory_id  stock_replenishment      type
14    AGDL67547                  9.0  Escaleta
DataFrame vazio encontrado após extrair a primeira linha:
   inventory_id  stock_replenishment           type
51    SJIT31691                  5.0  Suporte Sopro
DataFrame vazio encontrado após extrair a primeira linha:
   inventory_id  stock_replenishment       type
16    CMXV62869                  7.0  Percussão
DataFrame vazio encontrado após extrair a primeira linha:
   inventory_id  stock_replenishment       type
64    CIKE94068                  4.0  Cabo rolo
DataFrame vazio encontrado após extrair a primeira linha:
   inventory_id  stock_replenishment     type
60    XQUC46318                  6.0  Baqueta
DataFrame vazio encontrado após extrair a primeira linha:
   inven

In [144]:
result_dfs_list[0]

,inventory_id,stock_replenishment,type
0,JGED44177,9.0,Pandeiro
1,FXMN50419,70.0,Cabo
2,KRJP31426,4.0,Suporte Sopro
3,PSMV02965,28.0,Correia
4,TXWS20511,20.0,Percussão
5,WOXA67988,42.0,Escaleta
6,JJMG49643,8.0,Ukulele
7,MNSV53924,13.0,Cabo rolo
8,TPDH38567,25.0,Encordoamento
9,DSGP06979,15.0,Baqueta


In [145]:
result_dfs_list

[   inventory_id  stock_replenishment           type
 0     JGED44177                  9.0       Pandeiro
 1     FXMN50419                 70.0           Cabo
 2     KRJP31426                  4.0  Suporte Sopro
 3     PSMV02965                 28.0        Correia
 4     TXWS20511                 20.0      Percussão
 5     WOXA67988                 42.0       Escaleta
 6     JJMG49643                  8.0        Ukulele
 7     MNSV53924                 13.0      Cabo rolo
 8     TPDH38567                 25.0  Encordoamento
 9     DSGP06979                 15.0        Baqueta
 10    TAPQ85749                  5.0        Palheta
 11    WYFB54549                  8.0  Suporte Corda,
    inventory_id  stock_replenishment           type
 0     GLMK21133                  5.0       Pandeiro
 1     HFNJ52229                372.0           Cabo
 2     CJBY66878                  4.0  Suporte Sopro
 3     HSDO10298                 31.0        Correia
 4     STLG62905                  9.0      Pe

In [146]:
result_df

,inventory_id,ml_code,seller_sku,title,stock_replenishment,tiny_id,tiny_sku,qtd_item,qtd_to_send,type
0,LXQV54406,MLB2777157690,FULLERNIEBALLP04052,Correia P/ Violão Guitarra Polypro Marrom P040...,5.0,745431564,749699140528,1,5,Correia


### Contando estoque da WMS em Produtos sem estoque no período

In [147]:
df_wms = get_wms_data(SMARTGO_TOKEN)

In [148]:
df_wms

,id_depositante,depositante,idProduto,produto_nome,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
0,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056192,ALAVANCA DE GUITARRA FLOYD C/ROSCA CROMO (6 UN...,7899548645179,103CR,5
1,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056203,BAG PHX PERSONALIZADO P/ CONJ. PRATO E BAQUETA...,7899548630373,PAA130,77
2,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056204,Bateria Infantil Profissional Para Criança Tur...,7898475680628,TURBINHO-JZ,1
3,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056205,BATERIA PHX DISNEY INFANTIL PRINCESAS MOSAICO ...,7899548641836,BID-P1,2
4,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056206,"Bateria PHX Infantil 16"" Preta DS516C BK",7899548605579,DS516C BK,4
...,...,...,...,...,...,...,...
1712,4808,005 - Rogerio de Souza Instrumentos Musicais LTDA,1056964,Palheta,PALHETA2,None,832
1713,4808,005 - Rogerio de Souza Instrumentos Musicais LTDA,1056965,Porta Palheta,PPalheta,None,61
1714,4808,005 - Rogerio de Souza Instrumentos Musicais LTDA,1056958,Braçadeira instrumentos de corda Kits,Bracadeirakits,None,976
1715,5094,007 - LIVERPOOL INSTRUMENTOS E ACESSORIOS MUSI...,1114387,NIRVANA ECO BIO 7A MADEIRA,NI 7AM,7897937421007,1000


In [149]:
dic_column_name = {"ml_inventory_id": "inventory_id"}
df_no_itens = df_no_itens.rename(columns=dic_column_name)
df_sold_zero = df_sold_zero.rename(columns=dic_column_name)

In [150]:
df_tiny_fulfillment.sample()

,inventory_id,ml_code,SKU,Título do anúncio,ID Tiny,SKU Tiny,Quantidade do item,Tipo de produto
311,LCUY78585,MLB1951921857,FULLSG5145TP,3 Encordoamento Guitarra Triple Pack Sg 0.09 +...,695806136,7899574331794,1,Encordoamento


In [151]:
df_tiny_fulfillment_no_itens = pd.merge(
    df_no_itens, df_tiny_fulfillment, on="inventory_id", how="inner"
)

In [152]:
print(df_no_itens.shape)
print(df_tiny_fulfillment.shape)
print(df_tiny_fulfillment_no_itens.shape)

(72, 16)
(499, 8)
(122, 23)


In [153]:
df_tiny_fulfillment_no_itens.columns

Index(['ml_code_x', 'seller_sku', 'inventory_id', 'value_name', 'status',
       'title', 'available_quantity_today', 'transfer_status',
       'transfer_quantity', 'total_available_quantity', 'days_available',
       'total_sold_not_catalog', 'total_sold_catalog', 'total_sold',
       'period_send_fulfillment', 'stock_replenishment', 'ml_code_y', 'SKU',
       'Título do anúncio', 'ID Tiny', 'SKU Tiny', 'Quantidade do item',
       'Tipo de produto'],
      dtype='object')

In [154]:
# df_tiny_fulfillment_no_itens[['ml_code_x','ml_code_y']]
# df_tiny_fulfillment_no_itens[['ml_code_x','seller_sku','SKU', 'Título do anúncio']]

In [155]:
df_tiny_fulfillment_no_itens = df_tiny_fulfillment_no_itens.drop(
    columns=["ml_code_y", "seller_sku", "title"]
)

In [156]:
dic_new_names = {
    "ml_code_x": "ml_code",
    "SKU": "seller_sku",
    "Título do anúncio": "title",
    "Quantidade do item": "qtd_item",
    "ID Tiny": "tiny_id",
    "SKU Tiny": "tiny_sku",
    "Tipo de produto": "type",
}
df_tiny_fulfillment_no_itens = df_tiny_fulfillment_no_itens.rename(
    columns=dic_new_names
)

cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "type",
    "stock_replenishment",
    "status",
]

In [157]:
df_tiny_fulfillment_no_itens.sample()

,ml_code,inventory_id,value_name,status,available_quantity_today,transfer_status,transfer_quantity,total_available_quantity,days_available,total_sold_not_catalog,total_sold_catalog,total_sold,period_send_fulfillment,stock_replenishment,seller_sku,title,tiny_id,tiny_sku,qtd_item,type
93,MLB2003218577,WJIT31909,NaN,paused,0,None,NaN,0,0,0,0,0,0.0,0.0,7898529532477,Correia Guitarra Violão Baixo Basso Sf 105 Eu ...,746248920,RH-01BK,2,Correia


In [158]:
df_tiny_fulfillment_no_itens = df_tiny_fulfillment_no_itens[cols]
df_tiny_fulfillment_no_itens

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,type,stock_replenishment,status
0,OVRP11252,MLB1564998298,FULLDATALINKMICECOC,Cabo Microfone Balanceado Estereo Dmx Rolo Com...,754085190,7898571083767,1,Cabo rolo,0.0,paused
1,JJMG39331,MLB1440293157,FULLWINNER11014,Ukulele Soprano 21 Abs Tampo Sapele Vermelho ...,687918940,7899574332890,1,Ukulele,0.0,paused
2,LKXA44509,MLB1618249173,FULLTIMBRA8629C,Pandeiro 10 Profissional Aro Dourado Pele Pr...,616465636,7897626886292,1,Pandeiro,0.0,paused
3,SSGO43954,MLB1488916154,FULLTIMBRA8291C,Pandeiro 11 Profissional Prat Duplas Pele Pri...,616472756,7897626882911,1,Pandeiro,0.0,paused
4,ERIM51807,MLB949788598,P10LNINJA10FT,Cabo Santo Angelo Ninja L P10 X P10l - 10ft/3.05m,598796832,7899028824407,1,Cabo,0.0,paused
...,...,...,...,...,...,...,...,...,...,...
117,QTPB03410,MLB1992512780,None,Encordoamento 0.11 P/ Violão Aço 85/15 Bronze ...,730228363,7897626866874,1,Encordoamento,0.0,paused
118,VXCN97101,MLB1992506451,None,Encordoamento 0.11 P/ Violão Aço 85/15 Bronze ...,730228363,7897626866874,1,Encordoamento,0.0,paused
119,ITSG97062,MLB1990145508,None,Console Boxx U6 De Mistura 110v/220v,730222570,BOXXU6,1,Mesa,0.0,paused
120,ITDN89121,MLB1965264557,None,Microfone Lapela Tag TG-88LP,602491828,7898651941550,1,Microfone,0.0,paused


In [159]:
df_tiny_fulfillment_no_itens[df_tiny_fulfillment_no_itens["tiny_id"] == "1056269"]

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,type,stock_replenishment,status


In [160]:
df_wms.sample(2)

,id_depositante,depositante,idProduto,produto_nome,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
1299,4807,004 - M. A. Bueno Frederico Sonorizacao Ltda,1063649,Capa Violão Folk Luxo,a12,a12,94
783,4806,003 - Musical Center Ponta Grossa - Ltda - Filial,1056974,ENC. 010 LIGHT P/ GUITARRA C/ REVESTIMENTO OPT...,7899574320828,12092,60


In [161]:
df_wms_tf_no_itens_ci = pd.merge(
    df_tiny_fulfillment_no_itens,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoInterno",
    how='left'
)
df_wms_tf_no_itens_ce = pd.merge(
    df_tiny_fulfillment_no_itens,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoExterno",
    how='left'
)

In [195]:
print(df_wms_tf_no_itens_ci.shape)
print(df_wms_tf_no_itens_ce.shape)

(219, 17)
(25, 17)


In [196]:
df_wms_tf_no_itens_ci.sample()

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,type,stock_replenishment,status,id_depositante,depositante,idProduto,produto_nome,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
65,JKBQ98755,MLB1993044887,7891088010035,Correia Guitarra Violão Baixo Basso Cla 03 Cam...,565663877,PPalheta,6,Correia,0.0,paused,4805,002 - Musical Center Ponta Grossa - Ltda - Matriz,1057886,Porta Palheta,PPalheta,None,12


In [197]:
# df_junto = pd.merge(df_wms_tf_no_itens_ci, df_wms_tf_no_itens_ce, on='coluna_comum', how='outer')

df_wms_tf_no_itens = pd.concat(
    [df_wms_tf_no_itens_ci, df_wms_tf_no_itens_ce], ignore_index=True
)

print(df_wms_tf_no_itens.shape)

(244, 17)


In [198]:
df_wms_tf_no_itens = df_wms_tf_no_itens.drop_duplicates()

In [199]:
# df_wms_tf_no_itens.columns

In [200]:
print(df_wms_tf_no_itens.shape)

cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
    "quantidade_disponivel",
]

df_wms_tf_no_itens = df_wms_tf_no_itens[cols]
df_wms_tf_no_itens

(219, 17)


,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,stock_replenishment,status,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
0,ERIM51807,MLB949788598,P10LNINJA10FT,Cabo Santo Angelo Ninja L P10 X P10l - 10ft/3.05m,598796832,7899028824407,1,0.0,paused,7899028824407,4.5.20.232.330,3
1,ERIM51807,MLB949788598,P10LNINJA10FT,Cabo Santo Angelo Ninja L P10 X P10l - 10ft/3.05m,598796832,7899028824407,1,0.0,paused,7899028824407,4.5.20.232.330,11
2,FFIA52697,MLB1629321557,FULLP10LNINJA10FT,Cabo P10 X P10l Santo Angelo Ninja L 10ft - 3.05m,598796832,7899028824407,1,0.0,paused,7899028824407,4.5.20.232.330,3
3,FFIA52697,MLB1629321557,FULLP10LNINJA10FT,Cabo P10 X P10l Santo Angelo Ninja L 10ft - 3.05m,598796832,7899028824407,1,0.0,paused,7899028824407,4.5.20.232.330,11
4,DGBX50777,MLB949790387,P10LNINJA15FT,Cabo Santo Angelo Ninja P10 X P10l - 15ft/4.57m,598806000,7899028824414,1,0.0,paused,7899028824414,7899028824414,51
...,...,...,...,...,...,...,...,...,...,...,...,...
214,FNAF71120,MLB2192269456,FULLMEMPHISHONURD,Ukulele Soprano Memphis Tagima 21 Havaí Honu ...,736122808,7898651948795,1,0.0,active,7898651948795,HONU RD,1
215,FNAF71120,MLB2192269456,FULLMEMPHISHONURD,Ukulele Soprano Memphis Tagima 21 Havaí Honu ...,736122808,7898651948795,1,0.0,active,7898651948795,HONU RD,7
216,QTPB03410,MLB1992512780,None,Encordoamento 0.11 P/ Violão Aço 85/15 Bronze ...,730228363,7897626866874,1,0.0,paused,7897626866874,6687,600
217,VXCN97101,MLB1992506451,None,Encordoamento 0.11 P/ Violão Aço 85/15 Bronze ...,730228363,7897626866874,1,0.0,paused,7897626866874,6687,600


In [201]:
## Removendo e somando duplicatas
# Lista das colunas que devem ser usadas para identificar linhas repetidas
cols_to_check_duplicates = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
]

In [202]:
# # Identificar linhas duplicadas
# duplicated_rows = df_df_wms_tf_no_itens[df_wms_tf_sold_zero.duplicated(subset=cols_to_check_duplicates, keep=False)]

# Agrupar por linhas repetidas e somar a coluna quantidade_disponivel
df_df_wms_tf_no_itens_sum = (
    df_wms_tf_no_itens.groupby(cols_to_check_duplicates)["quantidade_disponivel"]
    .sum()
    .reset_index()
)

# organizando
cols_result = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
    "quantidade_disponivel",
]
df_df_wms_tf_no_itens_sum = df_df_wms_tf_no_itens_sum[cols_result]

In [203]:
# print(df_wms_tf_no_itens.shape)
# print(df_df_wms_tf_no_itens_sum.shape)

In [204]:
# df_wms_tf_no_itens[df_wms_tf_no_itens['inventory_id']=='ELKB12099']
# df_wms_tf_no_itens[df_wms_tf_no_itens['tiny_id']=='754085190']
# df_wms_tf_no_itens[df_wms_tf_no_itens['tiny_sku']=='7898571083767']

In [205]:
# df_df_wms_tf_no_itens_sum[df_df_wms_tf_no_itens_sum['inventory_id']=='ELKB12099']
# df_df_wms_tf_no_itens_sum[df_df_wms_tf_no_itens_sum['tiny_id']=='754085190']
# df_df_wms_tf_no_itens_sum[df_df_wms_tf_no_itens_sum['tiny_sku']=='7898571083767']

### Contando estoque da WMS em Produtos sem vendas no período

In [206]:
df_tiny_fulfillment_sold_zero = pd.merge(
    df_sold_zero, df_tiny_fulfillment, on="inventory_id", how="inner"
)

In [207]:
df_tiny_fulfillment_sold_zero.sample()

,ml_code_x,seller_sku,inventory_id,value_name,status,title,available_quantity_today,transfer_status,transfer_quantity,total_available_quantity,...,total_sold,period_send_fulfillment,stock_replenishment,ml_code_y,SKU,Título do anúncio,ID Tiny,SKU Tiny,Quantidade do item,Tipo de produto
30,MLB1418466676,NaN,BOWK32167,NaN,active,NaN,2,None,NaN,2,...,0,-2.0,0.0,MLB1418466676,FULLHERCULESDS543BB,Suporte Flauta Transversal Picollo Clarinete H...,558643843,7897626853218,1,Suporte Sopro


In [208]:
df_tiny_fulfillment_sold_zero.columns

Index(['ml_code_x', 'seller_sku', 'inventory_id', 'value_name', 'status',
       'title', 'available_quantity_today', 'transfer_status',
       'transfer_quantity', 'total_available_quantity', 'days_available',
       'total_sold_not_catalog', 'total_sold_catalog', 'total_sold',
       'period_send_fulfillment', 'stock_replenishment', 'ml_code_y', 'SKU',
       'Título do anúncio', 'ID Tiny', 'SKU Tiny', 'Quantidade do item',
       'Tipo de produto'],
      dtype='object')

In [209]:
# df_tiny_fulfillment_sold_zero[['ml_code_x','ml_code_y']]
# df_tiny_fulfillment_sold_zero[['ml_code_x','seller_sku','SKU', 'Título do anúncio']]

In [210]:
df_tiny_fulfillment_sold_zero = df_tiny_fulfillment_sold_zero.drop(
    columns=["ml_code_y", "seller_sku", "title"]
)

In [211]:
dic_new_names = {
    "ml_code_x": "ml_code",
    "SKU": "seller_sku",
    "Título do anúncio": "title",
    "Quantidade do item": "qtd_item",
    "ID Tiny": "tiny_id",
    "SKU Tiny": "tiny_sku",
    "Tipo de produto": "type",
}
df_tiny_fulfillment_sold_zero = df_tiny_fulfillment_sold_zero.rename(
    columns=dic_new_names
)

cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "type",
    "stock_replenishment",
    "status",
]

In [212]:
df_tiny_fulfillment_sold_zero.sample()

,ml_code,inventory_id,value_name,status,available_quantity_today,transfer_status,transfer_quantity,total_available_quantity,days_available,total_sold_not_catalog,total_sold_catalog,total_sold,period_send_fulfillment,stock_replenishment,seller_sku,title,tiny_id,tiny_sku,qtd_item,type
145,MLB1403791227,FVTQ20181,NaN,paused,2,transfer,1.0,3,1,0,0,0,-3.0,0.0,FULLTIMBRA8236,Surdo Axé Timbra 18 Pol X 40 Cm Em Alumínio Co...,558020144,7897626882362,1,Surdo


In [213]:
df_tiny_fulfillment_sold_zero = df_tiny_fulfillment_sold_zero[cols]

In [214]:
df_tiny_fulfillment_sold_zero.sample()

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,type,stock_replenishment,status
55,BOZK72331,MLB1937454357,FULLKIT02,Kit Alça Correia Almofadada + Straplock Roldan...,746248920,RH-01BK,2,Correia,0.0,active


In [216]:
df_wms_tf_sold_zero_ci = pd.merge(
    df_tiny_fulfillment_sold_zero,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoInterno",
    how='left'
)
df_wms_tf_sold_zero_ce = pd.merge(
    df_tiny_fulfillment_sold_zero,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoExterno",
    how='left'
)

In [217]:
print(df_wms_tf_sold_zero_ci.shape)
print(df_wms_tf_sold_zero_ce.shape)

(351, 17)
(171, 17)


In [218]:
# df_junto = pd.merge(df_wms_tf_no_itens_ci, df_wms_tf_no_itens_ce, on='coluna_comum', how='outer')

df_wms_tf_sold_zero = pd.concat(
    [df_wms_tf_sold_zero_ci, df_wms_tf_sold_zero_ce], ignore_index=True
)

print(df_wms_tf_sold_zero.shape)

(522, 17)


In [219]:
df_wms_tf_sold_zero = df_wms_tf_sold_zero.drop_duplicates()
print(df_wms_tf_sold_zero.shape)

(437, 17)


In [220]:
cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
    "quantidade_disponivel",
]

df_wms_tf_sold_zero = df_wms_tf_sold_zero[cols]
df_wms_tf_sold_zero.sample()

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,stock_replenishment,status,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
28,YAZB38863,MLB1440243016,FULLELIXIR3212,Encordoamento Elixir .012 Bronze 80/20 Nanoweb...,632364105,7897626832121,1,0.0,active,NaN,NaN,NaN


In [221]:
df_wms_tf_sold_zero[df_wms_tf_sold_zero["inventory_id"] == "ELKB12099"]
df_wms_tf_sold_zero[df_wms_tf_sold_zero["tiny_id"] == "754085190"]
df_wms_tf_sold_zero[df_wms_tf_sold_zero["tiny_sku"] == "7898571083767"]

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,stock_replenishment,status,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
3,ELKB12099,MLB1567957793,FULLDATALINK2MICECO,Cabo Microfone Balanceado Estereo Dmx Rolo Com...,754085190,7898571083767,2,0.0,active,NaN,NaN,NaN


In [222]:
# # Identificar linhas duplicadas
# duplicated_rows = df_wms_tf_sold_zero[df_wms_tf_sold_zero.duplicated(subset=cols_to_check_duplicates, keep=False)]

In [223]:
# duplicated_rows[duplicated_rows['inventory_id']=='ELKB12099']
# duplicated_rows[duplicated_rows['tiny_id']=='754085190']
# duplicated_rows[duplicated_rows['tiny_sku']=='7898571083767']

In [224]:
# Agrupar por linhas repetidas e somar a coluna quantidade_disponivel
df_wms_tf_sold_zero_sum = (
    df_wms_tf_sold_zero.groupby(cols_to_check_duplicates)["quantidade_disponivel"]
    .sum()
    .reset_index()
)

# organizando
cols_result = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
    "quantidade_disponivel",
]
df_wms_tf_sold_zero_sum = df_wms_tf_sold_zero_sum[cols_result]

In [225]:
print(df_wms_tf_sold_zero.shape)
print(df_wms_tf_sold_zero_sum.shape)

(437, 12)
(153, 12)


In [226]:
df_wms_tf_sold_zero_sum.sample()

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,stock_replenishment,status,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
67,MHXR70863,MLB1492560842,FULLPHXVIDMR1,Violão Profissional Phx Disney Infantil Mickey...,640794123,7899548663371,1,0.0,active,7899548663371,VID-MR1,21.0


In [227]:
df_wms_tf_sold_zero_sum[df_wms_tf_sold_zero_sum["inventory_id"] == "ELKB12099"]
df_wms_tf_sold_zero_sum[df_wms_tf_sold_zero_sum["tiny_id"] == "754085190"]
df_wms_tf_sold_zero_sum[df_wms_tf_sold_zero_sum["tiny_sku"] == "7898571083767"]

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,stock_replenishment,status,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
